In [1]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import io

import sddk

In [2]:
### configure session and groupurl
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [3]:
# request geojson with geographies
resp = requests.get("https://edh-www.adw.uni-heidelberg.de/download/edhGeographicData.json", headers={'User-Agent': ''})
resp

<Response [200]>

In [4]:
resp.json()["features"][:10]

[{'geometry': {'coordinates': [23.451654999, 38.497692999], 'type': 'Point'},
  'properties': {'ancient_findspot': 'Anthedon',
   'trismegistos_geo_uri': 'https://www.trismegistos.org/place/15743',
   'pleiades_uri': 'https://pleiades.stoa.org/places/540639',
   'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/0'},
  'type': 'Feature'},
 {'type': 'Feature',
  'properties': {'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/3',
   'ancient_findspot': 'Isthmia',
   'pleiades_uri': 'https://pleiades.stoa.org/places/570316',
   'trismegistos_geo_uri': 'https://www.trismegistos.org/place/15746'},
  'geometry': {'coordinates': [22.992664, 37.9158514], 'type': 'Point'}},
 {'type': 'Feature',
  'properties': {'trismegistos_geo_uri': 'https://www.trismegistos.org/place/1851',
   'ancient_findspot': 'Lebadea',
   'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/4'},
  'geometry': {'type': 'Point', 'coordinates': [23.273345, 38.221257]}},
 {'geometry': {'coordin

In [5]:
len(resp.json()["features"])

29656

In [5]:
geo_uri_dict = {} # dictionary arranged by uris
for feature in resp.json()["features"]:
    try:
        geo_uri_dict[feature["properties"]["uri"].rpartition("/")[2]] = feature
    except:
        pass

In [12]:
# check how it works
geo_uri_dict["0"]

{'geometry': {'coordinates': [23.451654999, 38.497692999], 'type': 'Point'},
 'properties': {'ancient_findspot': 'Anthedon',
  'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/0',
  'pleiades_uri': 'https://pleiades.stoa.org/places/540639',
  'trismegistos_geo_uri': 'https://www.trismegistos.org/place/15743'},
 'type': 'Feature'}

In [13]:
geo_uri_dict["0"]["geometry"]["coordinates"]

[23.451654999, 38.497692999]

In [15]:
sddk.write_file("SDAM_data/EDH/EDH_geo_dict_2020-09-15.json", geo_uri_dict, conf)

Your <class 'dict'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/EDH_geo_dict_2020-09-15.json"
